# Polymarket Copy Trading Analysis

Interactive analysis of copy trading performance using data from the SQLite database.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from datetime import datetime, timedelta

# Configure matplotlib
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data from Database

In [ ]:
# Connect to database
DB_PATH = '../../trades.db'

conn = sqlite3.connect(DB_PATH)

# Load all trades
df = pd.read_sql_query('SELECT * FROM trades ORDER BY timestamp_ms', conn)

# Convert timestamp to datetime
df['datetime'] = pd.to_datetime(df['timestamp_ms'], unit='ms')

print(f'Loaded {len(df):,} trades')
df.head()

## Basic Statistics

In [ ]:
print('=== Trading Statistics ===\n')
print(f'Total Trades:      {len(df):,}')
print(f'Unique Tokens:     {df["token_id"].nunique():,}')
print(f'Date Range:        {df["datetime"].min()} to {df["datetime"].max()}')
print(f'\nBuy Trades:        {len(df[df["side"] == "BUY"]):,}')
print(f'Sell Trades:       {len(df[df["side"] == "SELL"]):,}')
print(f'\nTotal Volume (USD): ${df["whale_usd"].sum():,.2f}')
print(f'Avg Trade Size:     ${df["whale_usd"].mean():.2f}')

## Trade Status Distribution

In [ ]:
# Simplify status for visualization
def simplify_status(status):
    if 'SKIPPED_SMALL' in status:
        return 'Skipped (Small)'
    elif 'SKIPPED_PROBABILITY' in status:
        return 'Skipped (Prob)'
    elif 'MOCK_ONLY' in status:
        return 'Mock Only'
    elif '200 OK' in status:
        return 'Executed'
    else:
        return 'Other'

df['status_simple'] = df['status'].apply(simplify_status)

# Plot status distribution
status_counts = df['status_simple'].value_counts()
fig, ax = plt.subplots(figsize=(10, 6))
status_counts.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Trade Status Distribution')
ax.set_xlabel('Status')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print('\nStatus Breakdown:')
for status, count in status_counts.items():
    pct = count / len(df) * 100
    print(f'  {status}: {count:,} ({pct:.1f}%)')

## Trading Volume Over Time

In [ ]:
# Daily volume
df['date'] = df['datetime'].dt.date
daily_volume = df.groupby('date')['whale_usd'].sum()

fig, ax = plt.subplots(figsize=(12, 6))
daily_volume.plot(kind='bar', ax=ax, color='green', alpha=0.7)
ax.set_title('Daily Trading Volume (USD)')
ax.set_xlabel('Date')
ax.set_ylabel('Volume (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Position Analysis

In [ ]:
# Load positions
positions_query = '''
SELECT 
    token_id,
    SUM(CASE WHEN side = 'BUY' THEN our_shares ELSE -our_shares END) as net_shares,
    SUM(CASE WHEN side = 'BUY' THEN our_usd ELSE 0 END) / 
        NULLIF(SUM(CASE WHEN side = 'BUY' THEN our_shares ELSE 0 END), 0) as avg_entry_price,
    COUNT(*) as trade_count
FROM trades
WHERE our_shares IS NOT NULL
GROUP BY token_id
HAVING ABS(net_shares) > 0.0001
ORDER BY ABS(net_shares) DESC
'''

positions = pd.read_sql_query(positions_query, conn)
print(f'Open Positions: {len(positions)}')
print(f'\nTop 10 Positions by Size:')
positions.head(10)

## Execution Rate Analysis

In [ ]:
# Calculate execution metrics
total_trades = len(df)
executed_trades = df['our_shares'].notna().sum()
execution_rate = executed_trades / total_trades * 100

print(f'=== Execution Metrics ===\n')
print(f'Total Whale Trades:  {total_trades:,}')
print(f'Executed Trades:     {executed_trades:,}')
print(f'Execution Rate:      {execution_rate:.1f}%')

# Pie chart
fig, ax = plt.subplots(figsize=(8, 8))
labels = ['Executed', 'Skipped']
sizes = [executed_trades, total_trades - executed_trades]
colors = ['#66b3ff', '#ff9999']
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax.set_title('Trade Execution Rate')
plt.tight_layout()
plt.show()

In [ ]:
# Close connection
conn.close()
print('Analysis complete!')